In [2]:
import nnunetv2
#setting the path

import os
os.environ["nnUNet_raw"] = "/home/cuixing/Foot&Ankle/Tarsus_split_rawdata"
os.environ["nnUNet_preprocessed"] = "/nfs/turbo/coe-mreedsensitive/Processing/Foot_and_Ankle/Xingyang_Cui/Data_Preprocessed"
os.environ["nnUNet_results"] = "/home/cuixing/Foot&Ankle/Results"  # 训练结果存放目录
# =Check
print("nnUNet_raw:", os.environ.get("nnUNet_raw"))
print("nnUNet_preprocessed:", os.environ.get("nnUNet_preprocessed"))
print("nnUNet_results:", os.environ.get("nnUNet_results"))


nnUNet_raw: /home/cuixing/Foot&Ankle/Tarsus_split_rawdata
nnUNet_preprocessed: /nfs/turbo/coe-mreedsensitive/Processing/Foot_and_Ankle/Xingyang_Cui/Data_Preprocessed
nnUNet_results: /home/cuixing/Foot&Ankle/Results


In [ ]:
#Method 1: directly training

!nnUNetv2_train 001 3d_fullres 0 \
  -tr nnUNetTrainer \
  -p nnUNetPlans \
 -pretrained_weights /nfs/turbo/coe-mreedsensitive/Processing/Foot_and_Ankle/SK/TS_FineTuning/nnUNet_results/Dataset072_TS_app_bones/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/checkpoint_best.pth



############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0
/home/cuixing/.local/lib/python3.9/site-packages/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py:164: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.grad_scaler = GradScaler() if self.device.type == 'cuda' else None

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#################################################################

2025-07-06 18:46:08.634090: do_dummy_2d_data_aug: False
2025-07-06 18:46:08.663750: Using splits from existing split file: /nfs/turbo/coe-mreedsensitive/Processing/Foot_and_Ankle/Xingyang_Cui/Data_Preprocessed/Dataset001_TS_app_bones/splits_final.json
2025-07-06 18:46:08.666704: The split file contains 1 splits.
2025-07-06 18:46:08.667614: Desired fold for training: 0
2025-07-06 18:46:08.668547: This split has 8 training and 4 validation cases.
using pin_memory on device 0
using pin_memory on device 0

This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [256, 96, 80], 'median_image_size_in_voxels': [1728.5, 533.0, 533.0], 'spacing': [0.75, 0.75, 0.75], 'normalization_schemes': ['CTNormalization'], 'use_mask_for_norm': [False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape', 'res

# Method 2:Frozen encoder layers

In [41]:
import json

plans_path = "/home/cuixing/Foot&Ankle/Results/Dataset001_TS_app_bones/nnUNetTrainer__nnUNetPlans__3d_fullres/plans.json"

with open(plans_path, 'r') as f:
    plans = json.load(f)

cfg = plans["configurations"]["3d_fullres"]

print("📊 plans.json 中 3d_fullres 配置的所有字段：\n")
for key, value in cfg.items():
    print(f"🔹 {key}: {value}")


📊 plans.json 中 3d_fullres 配置的所有字段：

🔹 data_identifier: nnUNetPlans_3d_fullres
🔹 preprocessor_name: DefaultPreprocessor
🔹 batch_size: 2
🔹 patch_size: [256, 96, 80]
🔹 median_image_size_in_voxels: [1728.5, 533.0, 533.0]
🔹 spacing: [0.75, 0.75, 0.75]
🔹 normalization_schemes: ['CTNormalization']
🔹 use_mask_for_norm: [False]
🔹 resampling_fn_data: resample_data_or_seg_to_shape
🔹 resampling_fn_seg: resample_data_or_seg_to_shape
🔹 resampling_fn_data_kwargs: {'is_seg': False, 'order': 3, 'order_z': 0, 'force_separate_z': None}
🔹 resampling_fn_seg_kwargs: {'is_seg': True, 'order': 1, 'order_z': 0, 'force_separate_z': None}
🔹 resampling_fn_probabilities: resample_data_or_seg_to_shape
🔹 resampling_fn_probabilities_kwargs: {'is_seg': False, 'order': 1, 'order_z': 0, 'force_separate_z': None}
🔹 architecture: {'network_class_name': 'dynamic_network_architectures.architectures.unet.PlainConvUNet', 'arch_kwargs': {'n_stages': 6, 'features_per_stage': [32, 64, 128, 256, 320, 320], 'conv_op': 'torch.nn.mo

📊 基本网络结构信息：


KeyError: 'spacing'

In [35]:
import os
import json
import torch
from nnunetv2.training.nnUNetTrainer.nnUNetTrainer import nnUNetTrainer

# 配置路径
ckpt_path = "/home/cuixing/Foot&Ankle/Results/Dataset001_TS_app_bones/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_best.pth"
plans_path = "/home/cuixing/Foot&Ankle/Results/Dataset001_TS_app_bones/nnUNetTrainer__nnUNetPlans__3d_fullres/plans.json"
dataset_json_path = "/nfs/turbo/coe-mreedsensitive/Processing/Foot_and_Ankle/Xingyang_Cui/Data_Preprocessed/Dataset001_TS_app_bones/dataset.json"

# 读取 JSON
with open(dataset_json_path, 'r') as f:
    dataset_json = json.load(f)

# ✅ 子类包装，提前赋值路径
class MyTrainer(nnUNetTrainer):
    def __init__(self, plans, configuration, fold, dataset_json):
        # 提前赋值路径（这些变量 super() 里要用）
        self.output_folder_base = "/home/cuixing/Foot&Ankle/Results/Dataset001_TS_app_bones"
        self.preprocessed_dataset_folder_base = "/nfs/turbo/coe-mreedsensitive/Processing/Foot_and_Ankle/Xingyang_Cui/Data_Preprocessed/Dataset001_TS_app_bones"
        super().__init__(plans, configuration, fold, dataset_json)

# 初始化并加载
trainer = MyTrainer(plans_path, "3d_fullres", 0, dataset_json)
trainer.initialize()

# 加载权重
ckpt = torch.load(ckpt_path, map_location="cpu")
trainer.network.load_state_dict(ckpt["network_weights"])

# 打印网络结构
print("📐 Network structure:\n")
print(trainer.network)


Using device: cuda:0


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [5]:
!nnUNetv2_train 001 3d_fullres 2 \
-tr FrozenEncoderTrainer \
-pretrained_weights /nfs/turbo/coe-mreedsensitive/Processing/Foot_and_Ankle/SK/TS_FineTuning/nnUNet_results/Dataset072_TS_app_bones/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/checkpoint_best.pth



############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Traceback (most recent call last):
  File "/home/cuixing/.local/bin/nnUNetv2_train", line 8, in <module>
    sys.exit(run_training_entry())
  File "/home/cuixing/.local/lib/python3.9/site-packages/nnunetv2/run/run_training.py", line 275, in run_training_entry
    run_training(args.dataset_name_or_id, args.configuration, args.fold, args.tr, args.p, args.pretrained_weights,
  File "/home/cuixing/.local/lib/python3.9/site-packages/nnunetv2/run/run_training.py", line 196, in run_training
    nnunet_trainer = get_trainer_from_args(dataset_name_or_id, configuration, fold, trainer_class_name,
  File "/home/cuixing/.local/lib/python3.9/site-packages/nnunetv2/run/run_training.py", line 43, in get_trainer_from_args
    r